In [1]:
from AOC_utils import get_day
import numpy as np

day = 19
input_data = get_day(day)

# print out first few lines to get a feel for the data
input_data[:7]

Downloaded day 19 input


['xkv{a>2504:A,x<1530:R,A}',
 'qz{a<1970:A,x>3109:A,a>2070:A,A}',
 'mxz{s>699:R,s>378:R,A}',
 'lhf{m<1064:drs,A}',
 'cnm{a>3392:hml,x<3423:nd,x<3797:vjg,nfn}',
 'tqg{a<247:A,x>3063:A,A}',
 'vsk{m<3711:R,x<3262:A,m<3859:A,A}']

In [3]:
example = '''
px{a<2006:qkq,m>2090:A,rfg}
pv{a>1716:R,A}
lnx{m>1548:A,A}
rfg{s<537:gd,x>2440:R,A}
qs{s>3448:A,lnx}
qkq{x<1416:A,crn}
crn{x>2662:A,R}
in{s<1351:px,qqz}
qqz{s>2770:qs,m<1801:hdj,R}
gd{a>3333:R,R}
hdj{m>838:A,pv}

{x=787,m=2655,a=1222,s=2876}
{x=1679,m=44,a=2067,s=496}
{x=2036,m=264,a=79,s=2244}
{x=2461,m=1339,a=466,s=291}
{x=2127,m=1623,a=2188,s=1013}
'''.split('\n')[1:-1]

In [2]:
len(input_data)

708

In [152]:
def parse_data(data):

    workflows = []
    parts = []
    line_break = False
    for line in data:
        if line == '':
            line_break = True
        elif line_break:
            parts.append(line)
        else:
            workflows.append(line)

    def parse_workflow(workflow):
        name, workflow = workflow.split('{')
        workflow = workflow.replace('}', '')
        conditions = workflow.split(',')
        else_condition = conditions[-1]
        conditions = conditions[:-1]
        conditions = [condition.split(':') for condition in conditions]
        parsed_conditions = []
        for condition in conditions:
            var = condition[0][0]
            operator = condition[0][1]
            value = int(condition[0][2:])
            parsed_conditions.append((var, operator, value, condition[1]))
        parsed_conditions.append(('x', '>', -9999, else_condition))
        return name, parsed_conditions
    
    workflow_dict = {}
    for workflow in workflows:
        name, conditions = parse_workflow(workflow)
        workflow_dict[name] = conditions

    def parse_part(part):
        new_part = {}
        part = part.replace('{', '').replace('}', '')
        part = part.split(',')
        for line in part:
            line = line.split('=')
            new_part[line[0]] = int(line[1])
        return new_part
    
    parts = [parse_part(part) for part in parts]

    # print(workflow_dict)
    # print(parts)

    total = 0
    for part in parts:
        output = 'in'
        while output != 'R' and output != 'A':
            for condition in workflow_dict[output]:
                var, operator, value, resp = condition
                if operator == '>':
                    if part[var] > value:
                        output = resp
                        break
                elif operator == '<':
                    if part[var] < value:
                        output = resp
                        break
        if output == 'A':
            total += sum(part.values())



    def part2_recursion(part_combo, workflow, wkf_idx):
        var, operator, value, resp = workflow[wkf_idx]
        var_keys = [key for key in part_combo.keys() if var in key.split('_')[1]]
        if wkf_idx == len(workflow) - 1:
            if resp == 'R':
                return []
            elif resp == 'A':
                return [part_combo]
            else:
                return part2_recursion(part_combo, workflow_dict[resp], 0)
        else:
            min_var = part_combo[var_keys[0]]
            max_var = part_combo[var_keys[1]]
            if operator == '>':
                if min_var > value:
                    if resp == 'R':
                        return []
                    elif resp == 'A':
                        return [part_combo]
                    else:
                        return part2_recursion(part_combo, workflow_dict[resp], 0)
                elif max_var > value:
                    new_part_combo = part_combo.copy()
                    new_part_combo[var_keys[0]] = value + 1
                    if resp == 'R':
                        high_side = []
                    elif resp == 'A':
                        high_side = [new_part_combo]
                    else:
                        high_side = part2_recursion(new_part_combo, workflow_dict[resp], 0)
                    new_part_combo2 = part_combo.copy()
                    new_part_combo2[var_keys[1]] = value
                    low_side = part2_recursion(new_part_combo2, workflow, wkf_idx+1)
                    return high_side + low_side
                else:
                    return part2_recursion(part_combo, workflow, wkf_idx+1)
            else: # operator == '<'
                if max_var < value:
                    if resp == 'R':
                        return []
                    elif resp == 'A':
                        return [part_combo]
                    else:
                        return part2_recursion(part_combo, workflow_dict[resp], 0)
                elif min_var < value:
                    new_part_combo = part_combo.copy()
                    new_part_combo[var_keys[1]] = value - 1
                    if resp == 'R':
                        low_side = []
                    elif resp == 'A':
                        low_side = [new_part_combo]
                    else:
                        low_side = part2_recursion(new_part_combo, workflow_dict[resp], 0)
                    new_part_combo2 = part_combo.copy()
                    new_part_combo2[var_keys[0]] = value
                    high_side = part2_recursion(new_part_combo2, workflow, wkf_idx+1)
                    return high_side + low_side
                else:
                    return part2_recursion(part_combo, workflow, wkf_idx+1)

    part_combinations = part2_recursion({'min_x': 1, 'max_x': 4000, 
                                         'min_m': 1, 'max_m': 4000, 
                                         'min_a': 1, 'max_a': 4000, 
                                         'min_s': 1, 'max_s': 4000},
                                        workflow_dict['in'], 0)

    total2 = 0
    for part_combo in part_combinations:
        values = np.array(list(part_combo.values())).reshape(-1, 2)
        values = np.abs(values[:, 0] - values[:, 1]).astype(np.int64) + 1
        total2 += np.prod(values)

    print("part 1:", total)
    print("part 2:", total2)

In [154]:
parse_data(input_data)

part 1: 434147
part 2: 136146366355609


In [153]:
parse_data(example)

part 1: 19114
part 2: 167409079868000
